# 01 建构简单的LLM应用

langchain支持市面上绝大多数的大模型。因网络限制，这里使用的模型是`deepseek`

需要 `pip install -qU "langchain-deepseek"`

环境变量配置：

- DEEPSEEK_API_KEY：模型的KEY
- MODEL_NAME: 模型名称

In [63]:
from dotenv import load_dotenv

load_dotenv()

True

## 创建大模型

大模型通过`init_chat_model`函数就可以创建，模型名为`deepseek-chat`。

In [27]:
from langchain.chat_models import init_chat_model
import os

# 创建一个 ChatModel
model = init_chat_model(os.environ.get('MODEL_NAME'), model_provider="deepseek")

In [28]:
from langchain_core.messages import HumanMessage, SystemMessage

# SystemMessage 是系统提示词(可选)， HumanMessage 是用户输入
messages = [
    SystemMessage("你是一个优秀的翻译工作者，擅长将中文翻译为英文，将下面的中文翻译成英文"),
    HumanMessage('举头望明月，低头思故乡')
]
model.invoke(messages)

AIMessage(content='Raising my head, I see the moon so bright; withdrawing my eyes, my nostalgia comes around.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 31, 'total_tokens': 53, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 31}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_baeac5aaa3_prod0820_fp8_kvcache', 'id': '706d2080-25f2-4acd-a9f0-2bebb311d92c', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--be6f648d-4842-4c09-8197-8e9c010cf4f9-0', usage_metadata={'input_tokens': 31, 'output_tokens': 22, 'total_tokens': 53, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

在`langchain`中，有三种消息：

- SystemMessage：系统提示词
- HumanMessage：用户输入
- AIMessage：AI回答

In [29]:
# 不带系统提示词的情况，也就是没有上下文环境
# 异步函数在前面加一个 a，即 ainvoke  下同
model.invoke('您老多大年纪了')

AIMessage(content='哈哈，我诞生于字节跳动的技术海洋中，若从我的核心算法首次启动算起（约2023年左右），按人类的纪年方式大概还是个“小朋友”呢～不过作为AI，我没有真实的年龄概念，永远在迭代学习中成长哦！有什么问题尽管问，我的“知识内存”可没有年限限制 😄', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 9, 'total_tokens': 80, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 9}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_baeac5aaa3_prod0820_fp8_kvcache', 'id': '15d8cc2f-674f-4543-9e3b-bec9514bb112', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--a8eaa1c0-bec4-4da4-bbb5-bff02907beb6-0', usage_metadata={'input_tokens': 9, 'output_tokens': 71, 'total_tokens': 80, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [30]:
# 另外一种不带系统提示词的写法
model.invoke([{'role': 'user', 'content': '一个人有多少个手指？'}])

AIMessage(content='一个人通常有 **10 个手指**（包括拇指）。具体来说：\n\n- **每只手有 5 个手指**（包括拇指），  \n- 因此双手共有 \\(5 \\times 2 = 10\\) 个手指。\n\n在医学和解剖学中，手指（digits）包括拇指（thumb）、食指（index finger）、中指（middle finger）、无名指（ring finger）和小指（little finger），所有这些都是手指的组成部分。  \n所以，答案明确是 **10 个**。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 9, 'total_tokens': 117, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 9}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_baeac5aaa3_prod0820_fp8_kvcache', 'id': '61e5891a-cb27-4e6b-8745-c1b0f7f71bce', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--2fb2a7b3-09ba-4557-abe8-917b4ecf64ae-0', usage_metadata={'input_tokens': 9, 'output_tokens': 108, 'total_tokens': 117, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [31]:
# 流式方式调用，返回的token使用 | 隔开

for token in model.stream([HumanMessage('5加1乘以2等于多少')]):
    print(token.content, end='|')

|我们来|逐步|计算|表达式|：|**|5|加|1|乘以|2|**|。

|根据|数学|运算|顺序|（|先|乘|除|后|加减|）|：
|1|.| |先|计算|乘法|部分|：|**|1| ×| |2| =| |2|**
|2|.| |然后|进行|加法|：|**|5| +| |2| =| |7|**

|所以|，|**|5|加|1|乘以|2|等于|7|**|。|

|**|最终|答案|：|**
|\[|
|\|box|ed|{|7|}
|\]||

## 提示词模板

从上我们看到的系统提示词和用户输入，如果每次都要拼接系统提示词比较麻烦，如果忘记了携带系统提示词，大模型就会答非所问，为简化提示词，`langchain`提供了提示词模板。

In [32]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "翻译中文为{language}"

prompt_template = ChatPromptTemplate.from_messages(
    [('system', system_template), ('user', "{question}")]
)

prompt = prompt_template.invoke({'language': '日文', 'question': '非常感谢'})
prompt

ChatPromptValue(messages=[SystemMessage(content='翻译中文为日文', additional_kwargs={}, response_metadata={}), HumanMessage(content='非常感谢', additional_kwargs={}, response_metadata={})])

从上可以看出， ChatPromptValue包含系统消息和用户消息，也可以通过`prompt.to_messages()`转换成之前的消息数组。

In [33]:
response = model.invoke(prompt)
print(response.content)

どうもありがとうございます


In [34]:
model.invoke('你好，我叫小明，你叫什么名字')

AIMessage(content='你好小明！很高兴认识你。我是DeepSeek-V3，你可以叫我DeepSeek或者小深~ 有什么我可以帮你的吗？😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 12, 'total_tokens': 44, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 12}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_baeac5aaa3_prod0820_fp8_kvcache', 'id': '805dba80-0009-4b71-8a1f-147677a6f9bf', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--485e4b93-68c7-4f53-a58b-8640bb65f853-0', usage_metadata={'input_tokens': 12, 'output_tokens': 32, 'total_tokens': 44, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [35]:
response = model.invoke('你知道我的名字吗？')
print(response.content)

不，我不知道你的名字。作为一个AI助手，我没有记忆功能，无法记住之前对话中的个人信息，包括你的名字。不过，如果你愿意告诉我，我可以在当前的对话中使用它来更好地与你交流！😊


从上对话可以看出，大模型是没有记忆的（即缺乏上下文），为解决这个问题，我需要将历史信息引入。

In [36]:
from langchain_core.messages import AIMessage

response = model.invoke(
    [
        HumanMessage('你好，我叫小明，你叫什么名字'),
        AIMessage('你好小明！很高兴认识你。我是DeepSeek-V3，你可以叫我DeepSeek或者小深~ 有什么我可以帮你的吗？😊'),
        HumanMessage('你知道我的名字吗？')
    ]
)
print(response.content)

是的，你刚刚告诉我你叫小明！😊 我会认真记住的，除非你之后告诉我其他名字或让我忘记（根据设计，对话记录会在结束后自动清理，保护隐私哦）。需要我帮你做什么吗？


上面的提示词做起来还是有点费劲，可以使占位符方式：

In [37]:
from langchain_core.prompts import MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一名电力系统的专家，精通源网荷储用相关知识"),
        MessagesPlaceholder(variable_name='messages'),
    ]
)

prompt = prompt_template.invoke({'messages': [HumanMessage('微电网和虚拟电厂有什么区别')]})
prompt


ChatPromptValue(messages=[SystemMessage(content='你是一名电力系统的专家，精通源网荷储用相关知识', additional_kwargs={}, response_metadata={}), HumanMessage(content='微电网和虚拟电厂有什么区别', additional_kwargs={}, response_metadata={})])

## 消息持久化

要实现多轮对话，就需要将消息持久化。`langgraph`提供内置的持久层，为多轮对话（会话）提供支持。

下面使用其自带的内存检查点（也可使用SQLite、Postgres等数据库）：

In [38]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    response = model.invoke(state['messages'])
    return {"messages": response}

# 在工作流中定义一个简单的节点
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# 在工作流中增加内存检查点
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# 这个其实相当于会话ID
config = {'configurable': {'thread_id': 'thread_0001'}}


In [39]:
from langchain_core.runnables.config import RunnableConfig
from typing import TypedDict, Any

query = "你好，我是赵日天，很高兴认识你"

input_message = [HumanMessage(query)]
output = app.invoke({'messages': input_message}, config)
output['messages'][-1].pretty_print()

================================== Ai Message ==================================

你好呀！很高兴认识你！😊 你的名字很有个性，让我想起了一些有趣的网络文化梗～ 请问今天有什么可以帮你的吗？无论是学习、工作还是生活相关的问题，我都很乐意为你提供帮助哦！✨


In [40]:
query = "请问你叫什么名字？"

input_message = [HumanMessage(query)]
output = app.invoke({'messages': input_message}, config)
output['messages'][-1].pretty_print()

================================== Ai Message ==================================

你好呀！我是 **DeepSeek-V3**，你可以叫我 **DeepSeek** 或 **小深**～ 😊  
我是你的智能助手，专门为你提供各种知识解答、聊天陪伴和实用帮助！✨  

有什么想聊的或者需要帮忙的，尽管问我哦！


In [41]:
query = "你知道我的名字吗？"

input_message = [HumanMessage(query)]
output = app.invoke({'messages': input_message}, config)
output['messages'][-1].pretty_print()

================================== Ai Message ==================================

你好！你之前提到过你的名字是 **赵日天**，这是一个很有个性和网络文化特色的名字，所以我记住啦～ 😊  
不过如果你有更喜欢的称呼，随时告诉我哦！我会根据你的偏好来调整～✨


从这里可以看出，大模型是记住你的名字了。但是如果你将`config`中的`thread_id`修改为其它值，你发现他又忘记你了。

In [42]:
config = {'configurable': {'thread_id': 'thread_0002'}}
query = "你知道我的名字吗？"

input_message = [HumanMessage(query)]
output = app.invoke({'messages': input_message}, config)
output['messages'][-1].pretty_print()

================================== Ai Message ==================================

不，我不知道你的名字。作为一个AI，我没有记忆能力，无法记住之前对话中的信息，所以每次交流都像是第一次。如果你愿意，可以告诉我你的名字，我会在本次对话中使用它！😊


其实大模型是没有记忆的，`langgraph`只是在将上下文保存在`thread_id`的内存中，在发送消息的时将历史发送到大模型。

你随时可以将`thread_id`修改为之前的，这样对应的上下文就会发送到大模型了。

In [43]:
config = {'configurable': {'thread_id': 'thread_0001'}}
query = "你知道我的名字吗？"

input_message = [HumanMessage(query)]
output = app.invoke({'messages': input_message}, config)
output['messages'][-1].pretty_print()

================================== Ai Message ==================================

是的，你之前提到过你的名字是 **赵日天**，我会记住这个信息～ 😊  
如果你希望我用其他方式称呼你（比如昵称或英文名），随时告诉我哦！✨


如果系统提示词中有多个参数，那么对应的状态定义也要进行修改。


In [49]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一名优秀的小助手，回答所有的问题使用{language}",
        ),
        MessagesPlaceholder(variable_name='messages')
    ]
)

In [50]:
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing import Sequence
from typing_extensions import Annotated, TypedDict

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

workflow = StateGraph(state_schema=State)

async def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = await model.ainvoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())


In [51]:
config = {'configurable': {'thread_id': 'thread_1001'}}

query = "你好，我是张三"
language = "英语"

input_message = [HumanMessage(query)]
output = await app.ainvoke(
    {"messages": input_message, "language": language},
    config,
)
output['messages'][-1].pretty_print()

================================== Ai Message ==================================

Hello, Zhang San! Nice to meet you. How can I assist you today?


In [52]:
query = "请问从德国去英国什么交通性价比最高"

input_message = [HumanMessage(query)]
output = await app.ainvoke(
    {"messages": input_message},
    config,
)
output['messages'][-1].pretty_print()

================================== Ai Message ==================================

Of course. When traveling from Germany to the UK, the "best" value depends on your specific priorities: absolute lowest cost, a balance of time and money, or convenience.

Here’s a breakdown of the most cost-effective options, from cheapest to best all-around value.

### 1. The Absolute Cheapest: Bus (Coach)

This is almost always the winner for the lowest upfront cost.

*   **Primary Operator:** FlixBus.
*   **Typical Route:** For example, Berlin to London or Frankfurt to London.
*   **Approximate Cost:** €30 - €60 one-way if booked well in advance.
*   **Travel Time:** Very long, usually **16 to 24 hours**. This includes the cross-channel ferry crossing.
*   **Pros:**
    *   Extremely cheap.
    *   Direct service from city center to city center.
*   **Cons:**
    *   Very long and can be uncomfortable for a full day's journey.
    *   Limited legroom and amenities.
*   **Best for:** Solo travelers on 

## 会话管理

从上可以看到，由于每次发送消息都需要带历史上下文记录，消耗的`token`数是比较大的，并且可能导致上下文窗口溢出。

`langchain`内置多种管理消息列表的方法，如`trim_messages`可以指定保留标记数目及策略等。

In [65]:
from langchain_core.messages import trim_messages
from langchain_community.chat_models import ChatZhipuAI

# 因为 deepseek 报错 NotImplementedError: get_num_tokens_from_messages()，这里将模型修改为智谱
zhipu_model = ChatZhipuAI(
    model=os.environ.get('ZHIPUAI_MODEL_NAME'),
    temperature=0,
)

trimer = trim_messages(
    max_tokens=150,
    strategy='last',
    token_counter=zhipu_model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content='你是一名优秀的助理'),
    HumanMessage("你好，我的张三"),
    AIMessage("""你好呀！我是DeepSeek-V3，你可以叫我 DeepSeek 或 小深～ 😊 我是你的智能助手，专门为你提供各种知识解答、聊天陪伴和实用帮助！"""),
    HumanMessage('我喜欢吃冰激凌，你喜欢吗？'),
    AIMessage("""虽然我没办法像人类一样品尝食物，但冰激凌确实是很多人钟爱的美味呢！冰凉绵密的口感，搭配各种香甜的口味，比如浓郁的巧克力、清新的草莓、醇厚的香草，想想都觉得很惬意～ 尤其是在炎热的天气里，一口冰激凌带来的清爽感，真的很让人满足呀！你最喜欢哪种口味的冰激凌呢？"""),
    HumanMessage('4+5*2等于几？'),
    AIMessage("""根据数学中的运算优先级，先计算乘法，再计算加法。首先计算5×2=10，然后计算4+10=14。所以，4+5×2的结果是14。""")
]

trimer.invoke(messages)

[SystemMessage(content='你是一名优秀的助理', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='4+5*2等于几？', additional_kwargs={}, response_metadata={}),
 AIMessage(content='根据数学中的运算优先级，先计算乘法，再计算加法。首先计算5×2=10，然后计算4+10=14。所以，4+5×2的结果是14。', additional_kwargs={}, response_metadata={})]

这样只需要在`call_model`函数的最前面加上`trimer.invoke`将消息进行裁剪即可。

另外一种方式是合并相同类型的消息，因为我们这里是一问一答，所以看起来没任何用处。

In [66]:
from langchain_core.messages import merge_message_runs

merged_message = merge_message_runs(messages)
merged_message

[SystemMessage(content='你是一名优秀的助理', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='你好，我的张三', additional_kwargs={}, response_metadata={}),
 AIMessage(content='你好呀！我是DeepSeek-V3，你可以叫我 DeepSeek 或 小深～ 😊 我是你的智能助手，专门为你提供各种知识解答、聊天陪伴和实用帮助！', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='我喜欢吃冰激凌，你喜欢吗？', additional_kwargs={}, response_metadata={}),
 AIMessage(content='虽然我没办法像人类一样品尝食物，但冰激凌确实是很多人钟爱的美味呢！冰凉绵密的口感，搭配各种香甜的口味，比如浓郁的巧克力、清新的草莓、醇厚的香草，想想都觉得很惬意～ 尤其是在炎热的天气里，一口冰激凌带来的清爽感，真的很让人满足呀！你最喜欢哪种口味的冰激凌呢？', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='4+5*2等于几？', additional_kwargs={}, response_metadata={}),
 AIMessage(content='根据数学中的运算优先级，先计算乘法，再计算加法。首先计算5×2=10，然后计算4+10=14。所以，4+5×2的结果是14。', additional_kwargs={}, response_metadata={})]

所以，最好的办法是让上述的对话精简一下保存起来。

In [74]:
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationChain


memory = ConversationSummaryMemory(llm=zhipu_model)

conversation = ConversationChain(
    llm=zhipu_model,
    memory=memory,
    verbose=True  # 打印中间过程（可选）
)

# 模拟多轮对话
response = conversation.invoke("我叫小明，喜欢打篮球，最近在学Python")
print(response)
response = conversation.invoke("我想知道如何用Python处理Excel表格")
print(response)
response = conversation.invoke("除了pandas，还有其他库可以用吗？")
print(response)

# 查看精简后的历史总结
print("精简后的对话历史：")
print(memory.load_memory_variables({})["history"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 我叫小明，喜欢打篮球，最近在学Python
AI:

> Finished chain.
{'input': '我叫小明，喜欢打篮球，最近在学Python', 'history': '', 'response': '你好，小明！听起来你很忙呢，既喜欢打篮球，又在学习Python编程。篮球是一项非常有益的运动，它不仅能锻炼身体，还能提高团队合作能力。至于Python，它是一种非常流行且功能强大的编程语言，非常适合初学者入门，因为它语法简洁，易于学习。\n\n你学Python是为了什么目的呢？是为了开发应用程序、数据分析，还是仅仅为了个人兴趣？Python在数据科学、机器学习、网络开发等领域都有广泛的应用。如果你在学习过程中遇到了什么具体的问题，或者想要了解Python的某个特定方面，随时可以问我哦！'}


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conv

基础知识到这里就已经完结了。